In [0]:
#Mounts drive to access google drive files
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Lets us save models to drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
#Lets us save models to drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
%tensorflow_version 1.x
import tensorflow
import datetime, os
print(tensorflow.__version__)
%load_ext tensorboard

In [0]:
#Launches TensroBoard
!kill 369
%tensorboard --logdir logs

In [0]:
#CNN Optimized
#Must restart runtime after each run
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import Precision, Recall
import time
import numpy as np
import sklearn
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

batch_size = 1000

import pickle

pickle_in = open("/content/drive/My Drive/Science Fair/Malaria Project/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/content/drive/My Drive/Science Fair/Malaria Project/y.pickle","rb")
y = pickle.load(pickle_in)

X = np.array(X)
y = np.array(y)

X = X/255.0

dense_layers = [1, 2, 3, 4]
layer_sizes = [64, 128, 256]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))


            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

            model.fit(X, y, batch_size=batch_size, epochs=26, validation_split=0.3, callbacks=[tensorboard])
            
           #Saving final model
            model.save('Final_Optimized_Model.h5')
            model_file = drive.CreateFile({'title' : 'Final_Optimized_Model.h5'})
            model_file.SetContentFile('Final_Optimized_Model.h5')
            model_file.Upload()
            drive.CreateFile({'id': model_file.get('id')})